# System Setup

**1. Install system dependencies and python packages. Prepare the environment.**

First, install all dependencies and python packages as `root`. Run commands and make sure the installations are successful.

```bash
apt update

apt install -y sudo locales wget tar tzdata git ccache cmake ninja-build build-essential llvm-11-dev clang-11 libiberty-dev libdwarf-dev libre2-dev libz-dev libssl-dev libboost-all-dev libcurl4-openssl-dev openjdk-8-jdk maven vim pip sysstat gcc-9 libjemalloc-dev nvme-cli curl zip unzip bison flex linux-tools-common linux-tools-generic linux-tools-`uname -r` mailutils

python3 -m pip install notebook==6.5.2
python3 -m pip install jupyter_server==1.23.4
python3 -m pip install jupyter_highlight_selected_word
python3 -m pip install jupyter_contrib_nbextensions
python3 -m pip install virtualenv==20.21.1
python3 -m pip uninstall -y ipython
python3 -m pip install ipython==8.21.0
python3 -m pip uninstall -y traitlets
python3 -m pip install traitlets==5.9.0
```

***Required for Ubuntu***

Check that there isn't an entry for your hostname mapped to 127.0.0.1 or 127.0.1.1 in /etc/hosts (Ubuntu is notorious for this). If there is, delete it.
Then add `<ip>` and `<hostname>` for master and worker nodes.

Example /etc/hosts:
 
```
127.0.0.1 localhost

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters

10.0.0.117 sr217
10.0.0.113 sr213
```

**2. Format and mount disks**

Create a python virtual environment to finish the system setup process:

```bash
virtualenv -p python3 -v venv
source venv/bin/activate
```

And install packages under `venv`:
```bash
(venv) python3 -m pip install questionary
```

Run script [init_disks.py](./init_disks.py) to format and mount disks. **Be careful when choosing the disks to format.** If you see errors like `device or resource busy`, perhaps the partition has been mounted, you should unmount it first. If you still see this error, reboot the system and try again.

Exit `venv`:
```bash
(venv) deactivate
```

**3. Create user `sparkuser`**

Create user `sparkuser` without password and with sudo priviledge. It's recommended to use one of the disks as the home directory instead of the system drive.

```bash
mkdir -p /data0/home/sparkuser
ln -s /data0/home/sparkuser /home/sparkuser
cp -r /etc/skel/. /home/sparkuser/
adduser --home /home/sparkuser --disabled-password --gecos "" sparkuser

chown -R sparkuser:sparkuser /data*

echo 'sparkuser ALL=(ALL:ALL) NOPASSWD:ALL' | EDITOR='tee -a' visudo
```

Generate ssh keys for `sparkuser`

```bashrc
su - sparkuser
```

```bashrc
rm -rf ~/.ssh
ssh-keygen -q -t rsa -N '' -f ~/.ssh/id_rsa <<<y >/dev/null 2>&1
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys

exit
```

Generate ssh keys for `root`, and enable no password ssh from `sparkuser`

```bash
rm -rf /root/.ssh
ssh-keygen -q -t rsa -N '' -f ~/.ssh/id_rsa <<<y >/dev/null 2>&1
cat /root/.ssh/id_rsa.pub >> /root/.ssh/authorized_keys
cat /home/sparkuser/.ssh/id_rsa.pub >> /root/.ssh/authorized_keys
```

Login to `sparkuser` and run the first-time ssh to the `root`

```bash
su - sparkuser
```

```bash
ssh -o StrictHostKeyChecking=no root@localhost ls
ssh -o StrictHostKeyChecking=no root@127.0.0.1 ls
ssh -o StrictHostKeyChecking=no root@`hostname` ls
```

***Required for Ubuntu***

Run below command to comment out lines starting from `If not running interactively, don't do anything` in ~/.bashrc

```bash
sed -i '5,9 s/^/# /' ~/.bashrc
```

**4. Configure jupyter notebook**

As `sparkuser`, install python packages

```bash
cd /home/sparkuser/.local/lib/ && rm -rf python*

python3 -m pip install --upgrade jsonschema
python3 -m pip install jsonschema[format]
python3 -m pip install sqlalchemy==1.4.46
python3 -m pip install papermill Black
python3 -m pip install NotebookScripter
python3 -m pip install findspark spylon-kernel matplotlib pandasql pyhdfs
python3 -m pip install ipywidgets jupyter_nbextensions_configurator ipyparallel
```


Configure jupyter notebook. Setup password when it prompts

```bash
jupyter notebook --generate-config

jupyter notebook password

mkdir -p ~/.jupyter/custom/

echo '.container { width:100% !important; }' >> ~/.jupyter/custom/custom.css

echo 'div.output_stderr { background: #ffdd; display: none; }'  >> ~/.jupyter/custom/custom.css

jupyter nbextension install --py jupyter_highlight_selected_word --user

jupyter nbextension enable highlight_selected_word/main

jupyter nbextension install --py widgetsnbextension --user

jupyter contrib nbextension install --user

jupyter nbextension enable codefolding/main

jupyter nbextension enable code_prettify/code_prettify

jupyter nbextension enable codefolding/edit

jupyter nbextension enable code_font_size/code_font_size

jupyter nbextension enable collapsible_headings/main

jupyter nbextension enable highlight_selected_word/main

jupyter nbextension enable ipyparallel/main

jupyter nbextension enable move_selected_cells/main

jupyter nbextension enable nbTranslate/main

jupyter nbextension enable scratchpad/main

jupyter nbextension enable tree-filter/index

jupyter nbextension enable comment-uncomment/main

jupyter nbextension enable export_embedded/main

jupyter nbextension enable hide_header/main

jupyter nbextension enable highlighter/highlighter

jupyter nbextension enable scroll_down/main

jupyter nbextension enable snippets/main

jupyter nbextension enable toc2/main

jupyter nbextension enable varInspector/main

jupyter nbextension enable codefolding/edit

jupyter nbextension enable contrib_nbextensions_help_item/main

jupyter nbextension enable freeze/main

jupyter nbextension enable hide_input/main

jupyter nbextension enable jupyter-js-widgets/extension

jupyter nbextension enable snippets_menu/main

jupyter nbextension enable table_beautifier/main

jupyter nbextension enable hide_input_all/main

jupyter nbextension enable spellchecker/main

jupyter nbextension enable toggle_all_line_numbers/main

jupyter nbextensions_configurator enable --user
```

Clone Gluten

```bash
cd ~
git clone https://github.com/apache/incubator-gluten.git gluten
```

Start jupyter notebook

```bash
mkdir -p ~/ipython
cd ~/ipython

nohup jupyter notebook --ip=0.0.0.0 --port=8888 &

find ~/gluten/tools/workload/benchmark_velox/ -maxdepth 1 -type f -exec cp {} ~/ipython \;
```

# Initialize
<font color=red size=3> Run this section after notebook restart! </font>

Specify datadir. The directories are used for spark.local.dirs and hadoop namenode/datanode.

In [ ]:
datadir=[f'/data{i}' for i in range(0, 8)]
datadir

Specify clients(workers). Leave it empty if the cluster is setup on the local machine.

In [ ]:
clients=''''''.split()
print(clients)

Specify JAVA_HOME

In [ ]:
java_home = '/usr/lib/jvm/java-8-openjdk-amd64'

In [ ]:
import os
import socket
import platform

user=os.getenv('USER')
print(f"user: {user}")
print()

masterip=socket.gethostbyname(socket.gethostname())
hostname=socket.gethostname()   
print(f"masterip: {masterip} hostname: {hostname}")
print()

hclients=clients.copy()
hclients.append(hostname)
print(f"master and workers: {hclients}")
print()


if clients:
    cmd = f"ssh {clients[0]} " + "\"lscpu | grep '^CPU(s)'\"" + " | awk '{print $2}'"
    client_cpu = !{cmd}
    cpu_num = client_cpu[0]

    cmd = f"ssh {clients[0]} " + "\"cat /proc/meminfo | grep MemTotal\"" + " | awk '{print $2}'"
    totalmemory = !{cmd}
    totalmemory = int(totalmemory[0])
else:
    cpu_num = os.cpu_count()
    totalmemory = !cat /proc/meminfo | grep MemTotal | awk '{print $2}'
    totalmemory = int(totalmemory[0])
    
print(f"cpu_num: {cpu_num}")
print()

print("total memory: ", totalmemory, "KB")
print()

mem_mib = int(totalmemory/1024)-1024
print(f"mem_mib: {mem_mib}")
print()

is_arm = platform.machine() == 'aarch64'
print("is_arm: ",is_arm)
print()

sparklocals=",".join([f'{l}/{user}/yarn/local' for l in datadir])
print(f"SPARK_LOCAL_DIR={sparklocals}")
print()

%cd ~

# Set up clients
<font color=red size=3> SKIP for single node </font>

## Install dependencies

Manually configure ssh login without password to all clients

```bash
ssh-copy-id -o StrictHostKeyChecking=no root@<client ip>
```

In [ ]:
for l in clients:
    !ssh root@{l} apt update > /dev/null 2>&1
    !ssh root@{l} apt install -y sudo locales wget tar tzdata git ccache cmake ninja-build build-essential llvm-11-dev clang-11 libiberty-dev libdwarf-dev libre2-dev libz-dev libssl-dev libboost-all-dev libcurl4-openssl-dev openjdk-8-jdk maven vim pip sysstat gcc-9 libjemalloc-dev nvme-cli curl zip unzip bison flex linux-tools-common linux-tools-generic linux-tools-`uname -r` > /dev/null 2>&1

## Create user

In [ ]:
for l in clients:
    !ssh -o StrictHostKeyChecking=no root@{l} ls

In [ ]:
for l in clients:
    !ssh root@{l} adduser --disabled-password --gecos '""' {user}

In [ ]:
for l in clients:
    !ssh root@{l} cp -r .ssh /home/{user}/
    !ssh root@{l} chown -R {user}:{user} /home/{user}/.ssh

In [ ]:
for l in clients:
    !ssh root@{l} "echo -e 'sparkuser ALL=(ALL:ALL) NOPASSWD:ALL' | EDITOR='tee -a' visudo"

***Required for Ubuntu***

Run below command to comment out lines starting from If not running interactively, don't do anything in ~/.bashrc

In [ ]:
for l in clients:
    !ssh {l} sed -i "'5,9 s/^/# /'" ~/.bashrc

Use /etc/hosts on master node

In [ ]:
for l in clients:
    !scp /etc/hosts root@{l}:/etc/hosts

## Setup disks

In [ ]:
for l in clients:
    !ssh root@{l} apt update > /dev/null 2>&1
    !ssh root@{l} apt install -y pip > /dev/null 2>&1
    !ssh root@{l} python3 -m pip install virtualenv

Manually run **2. Format and mount disks** section under [System Setup](#System-Setup)

# Configure Spark, Hadoop

## Download packages

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz > /dev/null 2>&1

In [ ]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.2.4/hadoop-3.2.4.tar.gz > /dev/null 2>&1
# backup url: !wget https://dlcdn.apache.org/hadoop/common/hadoop-3.2.4/hadoop-3.2.4.tar.gz > /dev/null 2>&1
if is_arm:
    # download both versions
    !wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5-aarch64.tar.gz > /dev/null 2>&1

## Create directories

In [ ]:
cmd=";".join([f"chown -R {user}:{user} " + l for l in datadir])
for l in hclients:
    !ssh root@{l} '{cmd}'

In [ ]:
cmd=";".join([f"rm -rf {l}/tmp; mkdir -p {l}/tmp" for l in datadir])
for l in hclients:
    !ssh {l} '{cmd}'

In [ ]:
cmd=";".join([f"mkdir -p {l}/{user}/hdfs/data; mkdir -p {l}/{user}/yarn/local" for l in datadir])
for l in hclients:
    !ssh {l} '{cmd}'

In [ ]:
!mkdir -p {datadir[0]}/{user}/hdfs/name
!mkdir -p {datadir[0]}/{user}/hdfs/namesecondary

In [ ]:
for l in hclients:
    !scp hadoop-3.2.4.tar.gz {l}:~/
    !scp spark-3.3.1-bin-hadoop3.tgz {l}:~/
    !ssh {l} "mv -f hadoop hadoop.bak; mv -f spark spark.bak"
    !ssh {l} "tar zxvf hadoop-3.2.4.tar.gz > /dev/null 2>&1"
    !ssh {l} "tar -zxvf spark-3.3.1-bin-hadoop3.tgz > /dev/null 2>&1"
    !ssh root@{l} "apt install -y openjdk-8-jdk > /dev/null 2>&1"
    !ssh {l} "ln -s hadoop-3.2.4 hadoop; ln -s spark-3.3.1-bin-hadoop3 spark"
    if is_arm:
        !ssh {l} "tar zxvf hadoop-3.3.5-aarch64.tar.gz > /dev/null 2>&1"
        !ssh {l} "cd hadoop && mv lib lib.bak && cp -rf ~/hadoop-3.3.5/lib ~/hadoop"

## Configure bashrc

In [ ]:

cfg=f'''export HADOOP_HOME=~/hadoop
export PATH=$PATH:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

export HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
export YARN_CONF_DIR=$HADOOP_HOME/etc/hadoop

export SPARK_HOME=~/spark
export PYTHONPATH=$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.9.5-src.zip:$PYTHONPATH
export PATH=$SPARK_HOME/bin:$PATH

'''

In [ ]:
if is_arm:
    cfg += 'export CPU_TARGET="aarch64"\nexport JAVA_HOME=/usr/lib/jvm/java-8-openjdk-arm64\nexport PATH=$JAVA_HOME/bin:$PATH\n'
else:
    cfg += f'export JAVA_HOME={java_home}\nexport PATH=$JAVA_HOME/bin:$PATH\n'

In [ ]:
with open("tmpcfg",'w') as f:
    f.writelines(cfg)

In [ ]:
for l in hclients:
    !scp tmpcfg {l}:~/tmpcfg.in
    !ssh {l} "cat ~/tmpcfg.in >> ~/.bashrc"

In [ ]:
for l in hclients:
    !ssh {l} tail -n10 ~/.bashrc

## Configure Hadoop

In [ ]:
for l in hclients:
    !ssh root@{l} "apt install -y libiberty-dev libxml2-dev libkrb5-dev libgsasl7-dev libuuid1 uuid-dev > /dev/null 2>&1"

### setup  short-circuit 

In [ ]:
for l in hclients:
    !ssh root@{l} "mkdir -p /var/lib/hadoop-hdfs/"
    !ssh root@{l} 'chown {user}:{user} /var/lib/hadoop-hdfs/'

### enable security.authorization

In [ ]:
coresite='''<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
      <name>fs.default.name</name>
      <value>hdfs://{:s}:8020</value>
      <final>true</final>
  </property>
  <property>
      <name>hadoop.security.authentication</name>
      <value>simple</value>
  </property>
  <property>
      <name>hadoop.security.authorization</name>
      <value>true</value>
  </property>
</configuration>
'''.format(hostname)

with open(f'/home/{user}/hadoop/etc/hadoop/core-site.xml','w') as f:
    f.writelines(coresite)
    
for l in clients:
    !scp ~/hadoop/etc/hadoop/core-site.xml {l}:~/hadoop/etc/hadoop/core-site.xml >/dev/null 2>&1

### set IP check, note the command <font color=red>", "</font>.join

In [ ]:
hadooppolicy='''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--

 Licensed to the Apache Software Foundation (ASF) under one
 or more contributor license agreements.  See the NOTICE file
 distributed with this work for additional information
 regarding copyright ownership.  The ASF licenses this file
 to you under the Apache License, Version 2.0 (the
 "License"); you may not use this file except in compliance
 with the License.  You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.

-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
    <name>security.service.authorization.default.hosts</name>
    <value>{:s}</value>
  </property>
  <property>
    <name>security.service.authorization.default.acl</name>
    <value>{:s} {:s}</value>
  </property>
  
  
    <property>
    <name>security.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for ClientProtocol, which is used by user code
    via the DistributedFileSystem.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.client.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for ClientDatanodeProtocol, the client-to-datanode protocol
    for block recovery.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for DatanodeProtocol, which is used by datanodes to
    communicate with the namenode.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.inter.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for InterDatanodeProtocol, the inter-datanode protocol
    for updating generation timestamp.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.namenode.protocol.acl</name>
    <value>*</value>
    <description>ACL for NamenodeProtocol, the protocol used by the secondary
    namenode to communicate with the namenode.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

 <property>
    <name>security.admin.operations.protocol.acl</name>
    <value>*</value>
    <description>ACL for AdminOperationsProtocol. Used for admin commands.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.refresh.user.mappings.protocol.acl</name>
    <value>*</value>
    <description>ACL for RefreshUserMappingsProtocol. Used to refresh
    users mappings. The ACL is a comma-separated list of user and
    group names. The user and group list is separated by a blank. For
    e.g. "alice,bob users,wheel".  A special value of "*" means all
    users are allowed.</description>
  </property>

  <property>
    <name>security.refresh.policy.protocol.acl</name>
    <value>*</value>
    <description>ACL for RefreshAuthorizationPolicyProtocol, used by the
    dfsadmin and mradmin commands to refresh the security policy in-effect.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.ha.service.protocol.acl</name>
    <value>*</value>
    <description>ACL for HAService protocol used by HAAdmin to manage the
      active and stand-by states of namenode.</description>
  </property>

  <property>
    <name>security.zkfc.protocol.acl</name>
    <value>*</value>
    <description>ACL for access to the ZK Failover Controller
    </description>
  </property>

  <property>
    <name>security.qjournal.service.protocol.acl</name>
    <value>*</value>
    <description>ACL for QJournalProtocol, used by the NN to communicate with
    JNs when using the QuorumJournalManager for edit logs.</description>
  </property>

  <property>
    <name>security.mrhs.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for HSClientProtocol, used by job clients to
    communciate with the MR History Server job status etc.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <!-- YARN Protocols -->

  <property>
    <name>security.resourcetracker.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceTrackerProtocol, used by the
    ResourceManager and NodeManager to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.resourcemanager-administration.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceManagerAdministrationProtocol, for admin commands.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationclient.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationClientProtocol, used by the ResourceManager
    and applications submission clients to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationmaster.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationMasterProtocol, used by the ResourceManager
    and ApplicationMasters to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.containermanagement.protocol.acl</name>
    <value>*</value>
    <description>ACL for ContainerManagementProtocol protocol, used by the NodeManager
    and ApplicationMasters to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.resourcelocalizer.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceLocalizer protocol, used by the NodeManager
    and ResourceLocalizer to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.job.task.protocol.acl</name>
    <value>*</value>
    <description>ACL for TaskUmbilicalProtocol, used by the map and reduce
    tasks to communicate with the parent tasktracker.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.job.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for MRClientProtocol, used by job clients to
    communciate with the MR ApplicationMaster to query job status etc.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationhistory.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationHistoryProtocol, used by the timeline
    server and the generic history service client to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  
  
  
  
</configuration>
'''.format((",").join(hclients),user,user)

with open(f'/home/{user}/hadoop/etc/hadoop/hadoop-policy.xml','w') as f:
    f.writelines(hadooppolicy)
    
for l in clients:
    !scp ~/hadoop/etc/hadoop/hadoop-policy.xml {l}:~/hadoop/etc/hadoop/hadoop-policy.xml >/dev/null 2>&1


### hdfs config, set replication to 1 to cache all the data

In [ ]:
hdfs_data=",".join([f'{l}/{user}/hdfs/data' for l in datadir])

hdfs_site=f'''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
    <name>dfs.namenode.secondary.http-address</name>
    <value>{hostname}:50090</value>
  </property>
  <property>
    <name>dfs.namenode.name.dir</name>
    <value>{datadir[0]}/{user}/hdfs/name</value>
    <final>true</final>
  </property>

  <property>
    <name>dfs.datanode.data.dir</name>
        <value>{hdfs_data}</value>
    <final>true</final>
  </property>

  <property>
    <name>dfs.namenode.checkpoint.dir</name>
    <value>{datadir[0]}/{user}/hdfs/namesecondary</value>
    <final>true</final>
  </property>
  <property>
    <name>dfs.name.handler.count</name>
    <value>100</value>
  </property>
  <property>
    <name>dfs.blocksize</name>
    <value>128m</value>
</property>
  <property>
    <name>dfs.replication</name>
    <value>1</value>
</property>

<property>
   <name>dfs.client.read.shortcircuit</name>
   <value>true</value>
</property>

<property>
   <name>dfs.domain.socket.path</name>
   <value>/var/lib/hadoop-hdfs/dn_socket</value>
</property>

</configuration>
'''


with open(f'/home/{user}/hadoop/etc/hadoop/hdfs-site.xml','w') as f:
    f.writelines(hdfs_site)
    
for l in clients:
    !scp ~/hadoop/etc/hadoop/hdfs-site.xml {l}:~/hadoop/etc/hadoop/hdfs-site.xml >/dev/null 2>&1


### mapreduce config

In [ ]:
mapreduce='''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>

     <property>
         <name>mapreduce.job.maps</name>
         <value>288</value>
     </property>
     <property>
         <name>mapreduce.job.reduces</name>
         <value>64</value>
     </property>

     <property>
         <name>mapreduce.map.java.opts</name>
         <value>-Xmx5120M -DpreferIPv4Stack=true</value>
     </property>
      <property>
         <name>mapreduce.map.memory.mb</name>
         <value>6144</value>
         </property>

     <property>
         <name>mapreduce.reduce.java.opts</name>
         <value>-Xmx5120M -DpreferIPv4Stack=true</value>
     </property>
     <property>
         <name>mapreduce.reduce.memory.mb</name>
         <value>6144</value>
     </property>
     <property>
         <name>yarn.app.mapreduce.am.staging-dir</name>
         <value>/user</value>
     </property>
     <property>
         <name>mapreduce.task.io.sort.mb</name>
         <value>2000</value>
     </property>
     <property>
         <name>mapreduce.task.timeout</name>
         <value>3600000</value>
     </property>
<!-- MapReduce Job History Server security configs -->
<property>
  <name>mapreduce.jobhistory.address</name>
  <value>{:s}:10020</value>
</property>

</configuration>
'''.format(hostname)


with open(f'/home/{user}/hadoop/etc/hadoop/mapred-site.xml','w') as f:
    f.writelines(mapreduce)
    
for l in clients:
    !scp ~/hadoop/etc/hadoop/mapred-site.xml {l}:~/hadoop/etc/hadoop/mapred-site.xml >/dev/null 2>&1


### yarn config

In [ ]:
yarn_site=f'''<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
  <property>
      <name>yarn.resourcemanager.hostname</name>
      <value>{hostname}</value>
  </property>
  <property>
      <name>yarn.resourcemanager.address</name>
      <value>{hostname}:8032</value>
  </property>
  <property>
      <name>yarn.resourcemanager.webapp.address</name>
      <value>{hostname}:8088</value>
  </property>
  <property>
      <name>yarn.nodemanager.resource.memory-mb</name>
      <value>{mem_mib}</value>
  </property>
  <property>
      <name>yarn.nodemanager.resource.cpu-vcores</name>
      <value>{cpu_num}</value>
  </property>
  <property>
      <name>yarn.nodemanager.pmem-check-enabled</name>
      <value>false</value>
  </property>

  <property>
      <name>yarn.nodemanager.vmem-check-enabled</name>
      <value>false</value>
  </property>
  <property>
      <name>yarn.nodemanager.vmem-pmem-ratio</name>
      <value>4.1</value>
  </property>
  <property>
      <name>yarn.nodemanager.aux-services</name>
      <value>mapreduce_shuffle,spark_shuffle</value>
  </property>

  <property>
      <name>yarn.scheduler.minimum-allocation-mb</name>
      <value>1024</value>
  </property>
  <property>
      <name>yarn.scheduler.maximum-allocation-mb</name>
      <value>{mem_mib}</value>
  </property>
  <property>
      <name>yarn.scheduler.minimum-allocation-vcores</name>
      <value>1</value>
  </property>
  <property>
      <name>yarn.scheduler.maximum-allocation-vcores</name>
      <value>{cpu_num}</value>
  </property>

  <property>
      <name>yarn.log-aggregation-enable</name>
      <value>false</value>
  </property>
  <property>
      <name>yarn.nodemanager.log.retain-seconds</name>
      <value>36000</value>
  </property>
  <property>
      <name>yarn.nodemanager.delete.debug-delay-sec</name>
      <value>3600</value>
  </property>
  <property>
      <name>yarn.log.server.url</name>
      <value>http://{hostname}:19888/jobhistory/logs/</value>
  </property>

  <property>
      <name>yarn.nodemanager.log-dirs</name>
      <value>/home/{user}/hadoop/logs/userlogs</value>
  </property>
  <property>
      <name>yarn.nodemanager.local-dirs</name>
      <value>{sparklocals}
      </value>
  </property>
  <property>
      <name>yarn.nodemanager.aux-services.spark_shuffle.class</name>
      <value>org.apache.spark.network.yarn.YarnShuffleService</value>
  </property>
</configuration>
'''


with open(f'/home/{user}/hadoop/etc/hadoop/yarn-site.xml','w') as f:
    f.writelines(yarn_site)
    
for l in clients:
    !scp ~/hadoop/etc/hadoop/yarn-site.xml {l}:~/hadoop/etc/hadoop/yarn-site.xml >/dev/null 2>&1


### hadoop-env

In [ ]:
#config java home
if is_arm:
    !echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-arm64" >> ~/hadoop/etc/hadoop/hadoop-env.sh
else:
    !echo "export JAVA_HOME={java_home}" >> ~/hadoop/etc/hadoop/hadoop-env.sh

for l in clients:
    !scp hadoop/etc/hadoop/hadoop-env.sh {l}:~/hadoop/etc/hadoop/ >/dev/null 2>&1


### workers config

In [ ]:
if clients:
    with open(f'/home/{user}/hadoop/etc/hadoop/workers','w') as f:
        f.writelines("\n".join(clients))
    for l in clients:
        !scp hadoop/etc/hadoop/workers {l}:~/hadoop/etc/hadoop/ >/dev/null 2>&1
else:
    !echo {hostname} > ~/hadoop/etc/hadoop/workers

### Copy jar from Spark for external shuffle service

In [ ]:
for l in hclients:
    !scp spark/yarn/spark-3.3.1-yarn-shuffle.jar {l}:~/hadoop/share/hadoop/common/lib/

## Configure Spark

In [ ]:
eventlog_dir=f'hdfs://{hostname}:8020/tmp/sparkEventLog'

In [ ]:
sparkconf=f'''
spark.eventLog.enabled    true
spark.eventLog.dir        {eventlog_dir}
spark.history.fs.logDirectory    {eventlog_dir}
'''

with open(f'/home/{user}/spark/conf/spark-defaults.conf','w+') as f:
    f.writelines(sparkconf)
    
for l in clients:
    !scp ~/spark/conf/spark-defaults.conf {l}:~/spark/conf/spark-defaults.conf >/dev/null 2>&1

In [ ]:
sparkenv = f'export SPARK_LOCAL_DIRS={sparklocals}\n'
with open(f'/home/{user}/.bashrc', 'a+') as f:
    f.writelines(sparkenv)
for l in clients:
    !scp ~/.bashrc {l}:~/.bashrc >/dev/null 2>&1

In [ ]:
for l in hclients:
    !ssh {l} tail -n10 ~/.bashrc

# Configure monitor & startups

In [ ]:
!cd ~
!git clone https://github.com/trailofbits/tsc_freq_khz.git

for l in clients:
    !scp -r tsc_freq_khz {l}:~/

for l in hclients:
    !ssh {l} 'cd tsc_freq_khz && make && sudo insmod ./tsc_freq_khz.ko' >/dev/null 2>&1
    !ssh root@{l} 'dmesg | grep tsc_freq_khz'

In [ ]:
startup=f'''#!/bin/bash
echo -1 > /proc/sys/kernel/perf_event_paranoid
echo 0 > /proc/sys/kernel/kptr_restrict
echo madvise >/sys/kernel/mm/transparent_hugepage/enabled
echo 1 > /proc/sys/kernel/numa_balancing
end=$(($(nproc) - 1))
for i in $(seq 0 $end); do echo performance > /sys/devices/system/cpu/cpu$i/cpufreq/scaling_governor; done
for file in $(find /sys/devices/system/cpu/cpu*/power/energy_perf_bias); do echo "0" > $file; done

if [ -d /home/{user}/sep_installed ]; then
    /home/{user}/sep_installed/sepdk/src/insmod-sep -g {user}
fi
'''

with open('/tmp/tmpstartup', 'w') as f:
    f.writelines(startup)

startup_service=f'''[Unit]
Description=Configure Transparent Hugepage, Auto NUMA Balancing, CPU Freq Scaling Governor

[Service]
ExecStart=/usr/local/bin/mystartup.sh

[Install]
WantedBy=multi-user.target
'''

with open('/tmp/tmpstartup_service', 'w') as f:
    f.writelines(startup_service)
    
for l in hclients:
    !scp /tmp/tmpstartup $l:/tmp/tmpstartup
    !scp /tmp/tmpstartup_service $l:/tmp/tmpstartup_service
    !ssh root@$l "cat /tmp/tmpstartup > /usr/local/bin/mystartup.sh"
    !ssh root@$l "chmod +x /usr/local/bin/mystartup.sh"
    !ssh root@$l "cat /tmp/tmpstartup_service > /etc/systemd/system/mystartup.service"
    !ssh $l "sudo systemctl enable mystartup.service"
    !ssh $l "sudo systemctl start mystartup.service"
    !ssh $l "sudo systemctl status mystartup.service"

## Install Emon

<font color=red size=3> Get the latest offline installer from [link](https://www.intel.com/content/www/us/en/developer/tools/oneapi/vtune-profiler-download.html?operatingsystem=linux&linux-install-type=offline) </font>

In [ ]:
offline_installer = 'https://registrationcenter-download.intel.com/akdlm/IRC_NAS/e7797b12-ce87-4df0-aa09-df4a272fc5d9/intel-vtune-2025.0.0.1130_offline.sh'
for l in hclients:
    !ssh {l} "wget {offline_installer} -q && chmod +x intel-vtune-2025.0.0.1130_offline.sh"

In [ ]:
for l in hclients:
    !ssh {l} "sudo ./intel-vtune-2025.0.0.1130_offline.sh -a -c -s --eula accept"

In [ ]:
for l in hclients:
    !ssh {l} "sudo chown -R {user}:{user} /opt/intel/oneapi/vtune/ && rm -f sep_installed && ln -s /opt/intel/oneapi/vtune/latest sep_installed"

In [ ]:
for l in hclients:
    !ssh {l} "cd sep_installed/sepdk/src/; echo -e \"\\n\\n\\n\" | ./build-driver"

In [ ]:
for l in hclients:
    !ssh root@{l} "/home/{user}/sep_installed/sepdk/src/rmmod-sep && /home/{user}/sep_installed/sepdk/src/insmod-sep -g {user}"

In [ ]:
for l in hclients:
    !ssh {l} "source /home/{user}/sep_installed/sep_vars.sh > /dev/null 2>&1; emon -v | head -n 1"

In [ ]:
for l in hclients:
    !ssh {l} 'echo "source /home/{user}/sep_installed/sep_vars.sh > /dev/null 2>&1" >> ~/.bashrc'

In [ ]:
for c in hclients:
    !ssh {c} 'tail -n1 ~/.bashrc'

## Inspect CPU Freq & HT

In [ ]:
if is_arm:
    t = r'''
    #include <stdlib.h>
    #include <stdio.h>
    #include <assert.h>
    #include <sys/time.h>
    #include <sched.h>
    #include <sys/shm.h>
    #include <errno.h>
    #include <sys/mman.h>
    #include <unistd.h>                        //used for parsing the command line arguments
    #include <fcntl.h>                        //used for opening the memory device file
    #include <math.h>                        //used for rounding functions
    #include <signal.h>
    #include <linux/types.h>
    #include <stdint.h>

    static inline uint64_t GetTickCount()
    {//Return ns counts
        struct timeval tp;
        gettimeofday(&tp,NULL);
        return tp.tv_sec*1000+tp.tv_usec/1000;
    }

    uint64_t CNT=CNT_DEF;

    int main()
    {

      uint64_t start, end;
      start=end=GetTickCount();

        asm  volatile (
          "1:\n"
          "SUBS %0,%0,#1\n"
          "bne 1b\n"
          ::"r"(CNT)
          );

      end=GetTickCount();

      printf(" total time = %lu, freq = %lu \n", end-start, CNT/(end-start)/1000);

      return 0;
    }
    '''
else:
    t=r'''
    #include <stdlib.h>
    #include <stdio.h>
    #include <assert.h>
    #include <sys/time.h>
    #include <sched.h>
    #include <sys/shm.h>
    #include <errno.h>
    #include <sys/mman.h>
    #include <unistd.h>                        //used for parsing the command line arguments
    #include <fcntl.h>                        //used for opening the memory device file
    #include <math.h>                        //used for rounding functions
    #include <signal.h>
    #include <linux/types.h>
    #include <stdint.h>

    static inline uint64_t GetTickCount()
    {//Return ns counts
        struct timeval tp;
        gettimeofday(&tp,NULL);
        return tp.tv_sec*1000+tp.tv_usec/1000;
    }

    uint64_t CNT=CNT_DEF;

    int main()
    {

      uint64_t start, end;
      start=end=GetTickCount();

        asm  volatile (
          "1:\n"
          "dec %0\n"
          "jnz 1b\n"
          ::"r"(CNT)
          );

      end=GetTickCount();

      printf(" total time = %lu, freq = %lu \n", end-start, CNT/(end-start)/1000);

      return 0;
    }
    '''

In [ ]:
%cd ~
with open("t.c", 'w') as f:
    f.writelines(t)
!gcc -O3 -DCNT_DEF=10000000000LL -o t t.c; gcc -O3 -DCNT_DEF=1000000000000LL -o t.delay t.c;
!for j in `seq 1 $(nproc)`; do echo -n $j; (for i in `seq 1 $j`; do taskset -c $i ./t.delay & done); sleep 1; ./t; killall t.delay; sleep 2; done

# <font color=red>Shutdown Jupyter; source ~/.bashrc; reboot Jupyter; run section [Initialize](#Initialize)</font>

# Build gluten

## Install docker

In [ ]:
# Instructions from https://docs.docker.com/engine/install/ubuntu/

# Add Docker's official GPG key:
!sudo -E apt-get update
!sudo -E apt-get install ca-certificates curl
!sudo -E install -m 0755 -d /etc/apt/keyrings
!sudo -E curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
!sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
!echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo -E tee /etc/apt/sources.list.d/docker.list > /dev/null
!sudo -E apt-get update

In [ ]:
!sudo -E apt-get install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin >/dev/null 2>&1

Configure docker proxy

In [ ]:
import os
http_proxy=os.getenv('http_proxy')
https_proxy=os.getenv('https_proxy')

if http_proxy or https_proxy:
    !sudo mkdir -p /etc/systemd/system/docker.service.d
    with open('/tmp/http-proxy.conf', 'w') as f:
        s = '''
[Service]
{}
{}
'''.format(f'Environment="HTTP_PROXY={http_proxy}"' if http_proxy else '', f'Environment="HTTPS_PROXY={https_proxy}"' if https_proxy else '')
        f.writelines(s)
    !sudo cp /tmp/http-proxy.conf /etc/systemd/system/docker.service.d
    
    !ssh root@localhost "mkdir -p /root/.docker"
    with open(f'/tmp/config.json', 'w') as f:
        s = f'''
{{
 "proxies": {{
   "default": {{
     "httpProxy": "{http_proxy}",
     "httpsProxy": "{https_proxy}",
     "noProxy": "127.0.0.0/8"
   }}
 }}
}}
        '''
        f.writelines(s)
    !ssh root@localhost "cp -f /tmp/config.json /root/.docker"

Configure maven proxy

In [ ]:
!mkdir -p ~/.m2

def get_proxy(proxy):
    pos0 = proxy.rfind('/')
    pos = proxy.rfind(':')
    host = http_proxy[pos0+1:pos]
    port = http_proxy[pos+1:]
    return host, port

if http_proxy or https_proxy:
    with open(f"/home/{user}/.m2/settings.xml","w+") as f:
        f.write('''<?xml version="1.0" encoding="UTF-8"?>
<settings xmlns="http://maven.apache.org/SETTINGS/1.2.0"
          xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
          xsi:schemaLocation="http://maven.apache.org/SETTINGS/1.2.0 https://maven.apache.org/xsd/settings-1.2.0.xsd">
    <proxies>''')
        if http_proxy:
            host, port = get_proxy(http_proxy)
            f.write(f'''
        <proxy>
          <id>http_proxy</id>
          <active>true</active>
          <protocol>http</protocol>
          <host>{host}</host>
          <port>{port}</port>
        </proxy>''')
        if https_proxy:
            host, port = get_proxy(http_proxy)
            f.write(f'''
        <proxy>
          <id>https_proxy</id>
          <active>true</active>
          <protocol>https</protocol>
          <host>{host}</host>
          <port>{port}</port>
        </proxy>''')
        f.write('''
    </proxies>
</settings>
''')

In [ ]:
!sudo systemctl daemon-reload

In [ ]:
!sudo systemctl restart docker.service

## Build gluten

In [ ]:
%cd ~

In [ ]:
import os
if not os.path.exists('gluten'):
    !git clone https://github.com/apache/incubator-gluten.git gluten

In [ ]:
# Build Arrow for the first time build.
!sed -i 's/--build_arrow=OFF/--build_arrow=ON/' ~/gluten/dev/package-vcpkg.sh

In [ ]:
%cd ~/gluten/tools/workload/benchmark_velox

In [ ]:
!bash build_gluten.sh

In [ ]:
for l in hclients:
    !scp ~/gluten/package/target/gluten-velox-bundle-spark*.jar {l}:~/

# Generate data

## Build spark-sql-perf

In [ ]:
!echo "deb https://repo.scala-sbt.org/scalasbt/debian all main" | sudo tee /etc/apt/sources.list.d/sbt.list
!echo "deb https://repo.scala-sbt.org/scalasbt/debian /" | sudo tee /etc/apt/sources.list.d/sbt_old.list
!curl -sL "https://keyserver.ubuntu.com/pks/lookup?op=get&search=0x2EE0EA64E40A89B84B2DF73499E82A75642AC823" | sudo apt-key add
!sudo -E apt-get update > /dev/null 2>&1
!sudo -E apt-get install sbt > /dev/null 2>&1

In [ ]:
import os
http_proxy=os.getenv('http_proxy')
https_proxy=os.getenv('https_proxy')

def get_proxy(proxy):
    pos0 = proxy.rfind('/')
    pos = proxy.rfind(':')
    host = http_proxy[pos0+1:pos]
    port = http_proxy[pos+1:]
    return host, port

sbt_opts=''

if http_proxy:
    host, port = get_proxy(http_proxy)
    sbt_opts = f'{sbt_opts} -Dhttp.proxyHost={host} -Dhttp.proxyPort={port}'
if https_proxy:
    host, port = get_proxy(https_proxy)
    sbt_opts = f'{sbt_opts} -Dhttps.proxyHost={host} -Dhttps.proxyPort={port}'
    
if sbt_opts:
    %env SBT_OPTS={sbt_opts}

In [ ]:
!git clone https://github.com/databricks/spark-sql-perf.git ~/spark-sql-perf

In [ ]:
!cd ~/spark-sql-perf && sbt package

In [ ]:
!cp ~/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar ~/ipython/

## Start Hadoop/Spark cluster, Spark history server

In [ ]:
!~/hadoop/bin/hadoop namenode -format

In [ ]:
!~/hadoop/bin/hadoop datanode -format 

In [ ]:
!~/hadoop/sbin/start-dfs.sh

In [ ]:
!hadoop dfsadmin -safemode leave

In [ ]:
!hadoop fs -mkdir -p /tmp/sparkEventLog

In [ ]:
!cd ~/spark && sbin/start-history-server.sh

In [ ]:
import os

master=''
if clients:
    !~/hadoop/sbin/start-yarn.sh
    master='yarn'
else:
    # If we run on single node, we use standalone mode
    !{os.environ['SPARK_HOME']}/sbin/stop-slave.sh
    !{os.environ['SPARK_HOME']}/sbin/stop-master.sh
    !{os.environ['SPARK_HOME']}/sbin/start-master.sh
    !{os.environ['SPARK_HOME']}/sbin/start-worker.sh spark://{hostname}:7077 -c {cpu_num}
    master=f'spark://{hostname}:7077'

In [ ]:
!jps
for l in clients:
    !ssh {l} jps

## TPCH

In [ ]:
!rm -rf ~/tpch-dbgen
!git clone https://github.com/databricks/tpch-dbgen.git ~/tpch-dbgen

In [ ]:
for l in clients:
    !scp -r ~/tpch-dbgen {l}:~/

In [ ]:
for l in hclients:
    !ssh {l} cd ~/tpch-dbgen && git checkout 0469309147b42abac8857fa61b4cf69a6d3128a8 && make clean && make OS=LINUX

In [ ]:
%cd  ~/gluten/tools/workload/tpch/gen_data/parquet_dataset

In [ ]:
# Suggest 2x cpu# partitions.
scaleFactor = 1500
numPartitions = 2*cpu_num if len(clients)==0 else len(clients)*2*cpu_num
dataformat = "parquet" # data format of data source
dataSourceCodec = "snappy"
rootDir = f"/tpch_sf{scaleFactor}_{dataformat}_{dataSourceCodec}" # root directory of location to create data in.

In [ ]:
# Verify parameters
print(f'scaleFactor = {scaleFactor}')
print(f'numPartitions = {numPartitions}')
print(f'dataformat = {dataformat}')
print(f'rootDir = {rootDir}')

In [ ]:
scala=f'''import com.databricks.spark.sql.perf.tpch._


val scaleFactor = "{scaleFactor}" // scaleFactor defines the size of the dataset to generate (in GB).
val numPartitions = {numPartitions}  // how many dsdgen partitions to run - number of input tasks.

val format = "{dataformat}" // valid spark format like parquet "parquet".
val rootDir = "{rootDir}" // root directory of location to create data in.
val dbgenDir = "/home/{user}/tpch-dbgen" // location of dbgen

val tables = new TPCHTables(spark.sqlContext,
    dbgenDir = dbgenDir,
    scaleFactor = scaleFactor,
    useDoubleForDecimal = false, // true to replace DecimalType with DoubleType
    useStringForDate = false) // true to replace DateType with StringType


tables.genData(
    location = rootDir,
    format = format,
    overwrite = true, // overwrite the data that is already there
    partitionTables = false, // do not create the partitioned fact tables
    clusterByPartitionColumns = false, // shuffle to get partitions coalesced into single files.
    filterOutNullPartitionValues = false, // true to filter out the partition with NULL key value
    tableFilter = "", // "" means generate all tables
    numPartitions = numPartitions) // how many dsdgen partitions to run - number of input tasks.
'''

with open("tpch_datagen_parquet.scala","w") as f:
    f.writelines(scala)

In [ ]:
executor_cores = 8
num_executors=cpu_num/executor_cores
executor_memory = (totalmemory - 10*1024*1024)/num_executors - 1*1024*1024

# Verify parameters
print(f'--master {master}')
print(f'--num-executors {int(num_executors)}')
print(f'--executor-cores {int(executor_cores)}')
print(f'--executor-memory {int(executor_memory)}k')
print(f'--conf spark.sql.shuffle.partitions={numPartitions}')
print(f'--conf spark.sql.parquet.compression.codec={dataSourceCodec}')

In [ ]:
tpch_datagen_parquet=f'''
cat tpch_datagen_parquet.scala | {os.environ['SPARK_HOME']}/bin/spark-shell \
  --master {master} \
  --name tpch_gen_parquet \
  --driver-memory 10g \
  --num-executors {int(num_executors)} \
  --executor-cores {int(executor_cores)} \
  --executor-memory {int(executor_memory)}k \
  --conf spark.executor.memoryOverhead=1g \
  --conf spark.sql.broadcastTimeout=4800 \
  --conf spark.driver.maxResultSize=4g \
  --conf spark.sql.shuffle.partitions={numPartitions} \
  --conf spark.sql.parquet.compression.codec={dataSourceCodec} \
  --conf spark.network.timeout=800s \
  --conf spark.executor.heartbeatInterval=200s \
  --jars /home/{user}/ipython/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar \
'''

with open("tpch_datagen_parquet.sh","w") as f:
    f.writelines(tpch_datagen_parquet)


In [ ]:
!nohup bash tpch_datagen_parquet.sh

## TPCDS

In [ ]:
!rm -rf ~/tpcds-kit
!git clone https://github.com/databricks/tpcds-kit.git ~/tpcds-kit

In [ ]:
for l in clients:
    !scp -r ~/tpcds-kit {l}:~/

In [ ]:
for l in hclients:
    !ssh {l} "cd ~/tpcds-kit/tools && make clean && make OS=LINUX CC=gcc-9"

In [ ]:
%cd ~/gluten/tools/workload/tpcds/gen_data/parquet_dataset

In [ ]:
# Suggest 2x cpu# partitions
scaleFactor = 1500
numPartitions = 2*cpu_num if len(clients)==0 else len(clients)*2*cpu_num
dataformat = "parquet" # data format of data source
dataSourceCodec = "snappy"
rootDir = f"/tpcds_sf{scaleFactor}_{dataformat}_{dataSourceCodec}" # root directory of location to create data in.

In [ ]:
# Verify parameters
print(f'scaleFactor = {scaleFactor}')
print(f'numPartitions = {numPartitions}')
print(f'dataformat = {dataformat}')
print(f'rootDir = {rootDir}')

In [ ]:
scala=f'''import com.databricks.spark.sql.perf.tpcds._

val scaleFactor = "{scaleFactor}" // scaleFactor defines the size of the dataset to generate (in GB).
val numPartitions = {numPartitions}  // how many dsdgen partitions to run - number of input tasks.

val format = "{dataformat}" // valid spark format like parquet "parquet".
val rootDir = "{rootDir}" // root directory of location to create data in.
val dsdgenDir = "/home/{user}/tpcds-kit/tools/" // location of dbgen

val tables = new TPCDSTables(spark.sqlContext,
    dsdgenDir = dsdgenDir,
    scaleFactor = scaleFactor,
    useDoubleForDecimal = false, // true to replace DecimalType with DoubleType
    useStringForDate = false) // true to replace DateType with StringType


tables.genData(
    location = rootDir,
    format = format,
    overwrite = true, // overwrite the data that is already there
    partitionTables = true, // create the partitioned fact tables
    clusterByPartitionColumns = true, // shuffle to get partitions coalesced into single files.
    filterOutNullPartitionValues = false, // true to filter out the partition with NULL key value
    tableFilter = "", // "" means generate all tables
    numPartitions = numPartitions) // how many dsdgen partitions to run - number of input tasks.
'''

with open("tpcds_datagen_parquet.scala","w") as f:
    f.writelines(scala)

In [ ]:
executor_cores = 8
num_executors=cpu_num/executor_cores
executor_memory = (totalmemory - 10*1024*1024)/num_executors - 1*1024*1024

# Verify parameters
print(f'--master {master}')
print(f'--num-executors {int(num_executors)}')
print(f'--executor-cores {int(executor_cores)}')
print(f'--executor-memory {int(executor_memory)}k')
print(f'--conf spark.sql.shuffle.partitions={numPartitions}')
print(f'--conf spark.sql.parquet.compression.codec={dataSourceCodec}')

In [ ]:
tpcds_datagen_parquet=f'''
cat tpcds_datagen_parquet.scala | {os.environ['SPARK_HOME']}/bin/spark-shell \
  --master {master} \
  --name tpcds_gen_parquet \
  --driver-memory 10g \
  --num-executors {int(num_executors)} \
  --executor-cores {int(executor_cores)} \
  --executor-memory {int(executor_memory)}k \
  --conf spark.executor.memoryOverhead=1g \
  --conf spark.sql.broadcastTimeout=4800 \
  --conf spark.driver.maxResultSize=4g \
  --conf spark.sql.shuffle.partitions={numPartitions} \
  --conf spark.sql.parquet.compression.codec={dataSourceCodec} \
  --conf spark.network.timeout=800s \
  --conf spark.executor.heartbeatInterval=200s \
  --jars /home/{user}/ipython/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar \
'''

with open("tpcds_datagen_parquet.sh","w") as f:
    f.writelines(tpcds_datagen_parquet)

In [ ]:
!nohup bash tpcds_datagen_parquet.sh

# Set up perf analysis tools (optional)

We have a set of perf analysis scripts under $GLUTEN_HOME/tools/workload/benchmark_velox/analysis. You can follow below steps to deploy the scripts on the same cluster and use them for performance analysis after each run.

## Install and deploy Trace-Viewer

Clone the master branch of project catapult:
```
cd ~
git clone https://github.com/catapult-project/catapult.git -b master
```

Trace-Viewer requires python version 2.7. Create a virtualenv for python2.7:
```
sudo apt install -y python2.7
virtualenv -p /usr/bin/python2.7 py27-env
source py27-env/bin/activate
```

Apply patch:

```
cd catapult
```
```
git apply <<EOF
diff --git a/catapult_build/dev_server.py b/catapult_build/dev_server.py
index a8b25c485..ec9638c7b 100644
--- a/catapult_build/dev_server.py
+++ b/catapult_build/dev_server.py
@@ -328,11 +328,11 @@ def Main(argv):

   app = DevServerApp(pds, args=args)

-  server = httpserver.serve(app, host='127.0.0.1', port=args.port,
+  server = httpserver.serve(app, host='0.0.0.0', port=args.port,
                             start_loop=False, daemon_threads=True)
   _AddPleaseExitMixinToServer(server)
   # pylint: disable=no-member
-  server.urlbase = 'http://127.0.0.1:%i' % server.server_port
+  server.urlbase = 'http://0.0.0.0:%i' % server.server_port
   app.server = server

   sys.stderr.write('Now running on %s\n' % server.urlbase)
EOF
```

Start the service:

```
mkdir -p ~/trace_result
cd ~/catapult && nohup ./bin/run_dev_server --no-install-hooks -d ~/trace_result -p1088 &
```

## Deploy perf analysis scripts

Create a virtualenv to run the perf analaysis scripts:

In [ ]:
%cd ~

In [ ]:
!virtualenv -p python3 -v paus-env
!source ~/paus-env/bin/activate && python3 -m pip install -r ~/gluten/tools/workload/benchmark_velox/analysis/requirements.txt

We will put all perf analysis notebooks under `$HOME/PAUS`. Create the directory and start the notebook:

In [ ]:
!mkdir -p ~/PAUS && cd ~/PAUS && source ~/paus-env/bin/activate && nohup jupyter notebook --ip=0.0.0.0 --port=8889 &

Package the virtual environment so that it can be distributed to other nodes:

In [ ]:
!cd ~ && tar -czf paus-env.tar.gz paus-env

Distribute to the worker nodes:

In [ ]:
for l in clients:
    !scp ~/paus-env.tar.gz {l}:~/
    !ssh {l} tar -zxf paus-env.tar.gz